In [54]:
# prepare
import numpy as np
import pandas as pd
import sys
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

seed = 782
np.random.seed(seed)
from sklearn.preprocessing import Normalizer

In [55]:
# load data
train_df= pd.read_csv("./datasets/train.csv")
train = train_df.values
test = pd.read_csv("./datasets/test.csv").values

In [56]:
X_train = train[:, 1:]
y_train = train[:, 0]

In [57]:
def normalization(x, mu, sigma):
    
    x_norm = np.zeros_like(x)

    for n in range(len(x)):
        for j in range(len(x[n])):
            if(sigma[j]!=0):
                x_norm[n,j] = (x[n,j] - mu[j]) / sigma[j]
            else:
                x_norm[n,j] = 0
                    
    return x_norm

In [58]:
mu = np.mean(X_train, axis = 0)
sigma = np.max(X_train, axis=0) - np.min(X_train, axis=0)

In [59]:
# test = normalization(test, mu, sigma)
normalizer = Normalizer(norm='max')
X_train = normalizer.fit_transform(X_train)
test = normalizer.transform(test)

In [63]:
y_train = pd.get_dummies(y_train).values

In [64]:
# 定义模型
def Relu(x, derivative=False):
    if derivative == False:
        return x * (x > 0)
    else:
        return 1 * (x > 0)

In [65]:
def Softmax(x):
    x -= np.max(x)
    sm = (np.exp(x).T / np.sum(np.exp(x),axis=1)).T
    return sm

In [115]:
def createWeightAndbiases():
    n_inputs = 28 * 28
    hidden1 = 30
    n_outputs = 10
    
    # layer1
    w1 = np.random.normal(0, n_inputs ** -0.5, [n_inputs, hidden1])
    b1 = np.random.normal(0, n_inputs ** -0.5, [1, hidden1])
    
    w2 = np.random.normal(0, hidden1 ** -0.5, [hidden1, n_outputs])
    b2 = np.random.normal(0, hidden1 ** -0.5, [1, n_outputs])
    
    return [w1, w2, b1, b2]

In [178]:
# dorpout
def Dropout(x, dropout_percent):
    data = [np.ones_like(x)]
    mask = np.random.binomial( data, (1 - dropout_percent) )[0] / (1 - dropout_percent)  
    return mask

In [187]:
# predict
def predict(weights, X, dropout_percent=0):
    w1, w2, b1, b2 = weights
    
    first = Relu(np.dot(x, w1) + b1)
    
    return [first, Softmax(np.dot(first, w2) + b2)]


In [188]:
# 衡量精确度
def accuracy(output, y):
    hit = 0
    output = np.argmax(output, axis = 1)
    y = np.argmax(y, axis=1)
    for x, y in zip(output, y):
        if(x == y):
            hit += 1
    p = (hit * 100) / output.shape[0]
    return p

In [190]:
def log2(x):
    if x != 0:
        return np.log(x)
    else:
        return -np.inf

def log(y):
    return [[log2(nx) for nx in x] for x in y]

def cost(Y_predict, Y_right, weights, nabla):
    w1, w2, b1, b2 = weights
    weights_sum_square = np.mean(w1 ** 2) + np.mean(w2 ** 2)
    Loss = -np.mean(Y_right * log(Y_predict) + (1 - Y_right) * log(1 - Y_predict) + nabla / 2 * weights_sum_square)
    return Loss